In [1]:
import tensorflow as tf
import tensorflowvisu
import math

#if you want to have animation, you need to have ffmpeg installed on your computer
from matplotlib import animation, rc
from IPython.display import HTML
rc('animation', html='html5')

from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
tf.set_random_seed(0)

In [2]:
# Download images and labels into mnist.test (10K images+labels) and mnist.train (60K images+labels)
mnist = read_data_sets("data", one_hot=True, reshape=False, validation_size=0)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [3]:
# input X: 28x28 grayscale images, the first dimension (None) will index the images in the mini-batch
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
# correct answers will go here
Y_ = tf.placeholder(tf.float32, [None, 10])

print("X: " + str(X))
print("Y_: " + str(Y_))

X: Tensor("Placeholder:0", shape=(?, 28, 28, 1), dtype=float32)
Y_: Tensor("Placeholder_1:0", shape=(?, 10), dtype=float32)


In [4]:
# flatten the images into a single line of pixels
# -1 in the shape definition means "the only possible dimension that will preserve the number of elements"
XX = tf.reshape(X, [-1, 784])
# weights W[784, 10]   784=28*28
W = tf.Variable(tf.zeros([784, 10]))
# biases b[10]
b = tf.Variable(tf.zeros([10]))
# The model
Y = tf.nn.softmax(tf.matmul(XX, W) + b)

print("XX: " + str(XX))
print("W: " + str(W))
print("b: " + str(b))
print("Y: " + str(Y))

XX: Tensor("Reshape:0", shape=(?, 784), dtype=float32)
W: Tensor("Variable/read:0", shape=(784, 10), dtype=float32)
b: Tensor("Variable_1/read:0", shape=(10,), dtype=float32)
Y: Tensor("Softmax:0", shape=(?, 10), dtype=float32)


In [5]:
# loss function: cross-entropy = - sum( Y_i * log(Yi) )
#                           Y: the computed output vector
#                           Y_: the desired output vector

# cross-entropy
# log takes the log of each element, * multiplies the tensors element by element
# reduce_mean will add all the components in the tensor
# so here we end up with the total cross-entropy for all images in the batch
cross_entropy = -tf.reduce_mean(Y_ * tf.log(Y)) * 1000.0  # normalized for batches of 100 images,
                                                          # *10 because  "mean" included an unwanted division by 10

# accuracy of the trained model, between 0 (worst) and 1 (best)
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("cross_entropy: " + str(cross_entropy))
print("correct_prediction: " + str(correct_prediction))
print("accuracy: " + str(accuracy))

cross_entropy: Tensor("mul_1:0", shape=(), dtype=float32)
correct_prediction: Tensor("Equal:0", shape=(?,), dtype=bool)
accuracy: Tensor("Mean_1:0", shape=(), dtype=float32)


In [6]:
# training, learning rate = 0.005
train_step = tf.train.GradientDescentOptimizer(0.005).minimize(cross_entropy)

print("train_step: " + str(train_step))

train_step: name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_Variable/ApplyGradientDescent"
input: "^GradientDescent/update_Variable_1/ApplyGradientDescent"



In [7]:
# init
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

print("sess: " + str(sess))

sess: <tensorflow.python.client.session.Session object at 0x115239828>


In [8]:
# You can call this function in a loop to train the model, 100 images at a time
def training_step(i, update_test_data, update_train_data):

    # training on batches of 100 images with 100 labels
    batch_X, batch_Y = mnist.train.next_batch(100)

    # compute training values for visualisation
    if update_train_data:
        a, c, im, w, b = sess.run([accuracy, cross_entropy, I, allweights, allbiases], feed_dict={X: batch_X, Y_: batch_Y})
        datavis.append_training_curves_data(i, a, c)
        datavis.append_data_histograms(i, w, b)
        datavis.update_image1(im)
        print(str(i) + ": accuracy:" + str(a) + " loss: " + str(c))

    # compute test values for visualisation
    if update_test_data:
        a, c, im = sess.run([accuracy, cross_entropy, It], feed_dict={X: mnist.test.images, Y_: mnist.test.labels})
        datavis.append_test_curves_data(i, a, c)
        datavis.update_image2(im)
        print(str(i) + ": ********* epoch " + str(i*100//mnist.train.images.shape[0]+1) + " ********* test accuracy:" + str(a) + " test loss: " + str(c))

    # the backpropagation training step
    sess.run(train_step, feed_dict={X: batch_X, Y_: batch_Y})

In [9]:
# matplotlib visualisation
allweights = tf.reshape(W, [-1])
allbiases = tf.reshape(b, [-1])
I = tensorflowvisu.tf_format_mnist_images(X, Y, Y_)  # assembles 10x10 images by default
It = tensorflowvisu.tf_format_mnist_images(X, Y, Y_, 1000, lines=25)  # 1000 images on 25 lines
datavis = tensorflowvisu.MnistDataVis()

animation = datavis.create_animation(training_step, iterations=100+1, train_data_update_freq=10, test_data_update_freq=50, more_tests_at_start=True)
HTML(animation.to_html5_video()) #bug : no video on github

0: accuracy:0.13 loss: 230.259
0: ********* epoch 1 ********* test accuracy:0.098 test loss: 230.26
10: accuracy:0.6 loss: 115.933
10: ********* epoch 1 ********* test accuracy:0.6787 test loss: 104.211
20: accuracy:0.83 loss: 61.2917
20: ********* epoch 1 ********* test accuracy:0.8488 test loss: 59.6142
30: accuracy:0.81 loss: 63.7258
30: ********* epoch 1 ********* test accuracy:0.8639 test loss: 52.2593
40: accuracy:0.88 loss: 45.9763
40: ********* epoch 1 ********* test accuracy:0.8704 test loss: 48.1346
50: accuracy:0.88 loss: 46.4928
50: ********* epoch 1 ********* test accuracy:0.8728 test loss: 46.7755
60: accuracy:0.95 loss: 28.2206
70: accuracy:0.86 loss: 53.9742
80: accuracy:0.89 loss: 31.6543
90: accuracy:0.92 loss: 31.7568
101: accuracy:0.91 loss: 38.8968
101: ********* epoch 1 ********* test accuracy:0.8812 test loss: 41.3588
